
# Setup


## Download the required libraries

In [ ]:
import sys

print( sys.executable )

!"{sys.executable}" -u -m pip install numpy
!"{sys.executable}" -u -m pip install pandas
!"{sys.executable}" -u -m pip install matplotlib
!"{sys.executable}" -u -m pip install tabulate



## Import the required libraries


In [ ]:
import numpy as numpy
import pandas as pandas
import matplotlib.pyplot as plt

from IPython.display import HTML, display
import tabulate



## Define Useful Function

In [ ]:

def displayAsTable( lst: list[list] ) -> None:
    display(
        HTML(
            tabulate.tabulate(lst, tablefmt='html')
        )
    )



# Exploratory Data Analysis


## Load the datasets and combine them into one

These data sets exist in four separate files USDA website only lets us download a maximum number of rows of 50,000 per file.

In [ ]:

df_1 = pandas.read_csv (r'data/CropYields-1909-1939.csv')
df_2 = pandas.read_csv (r'data/CropYields-1940-1969.csv')
df_3 = pandas.read_csv (r'data/CropYields-1970-1989.csv')
df_4 = pandas.read_csv (r'data/CropYields-1990-2007.csv')

df = pandas.concat([df_1, df_2, df_3, df_4])




## We take a look at the column names and the first couple of rows


In [ ]:
df.head()


We can see that there seems to be meta-data about this data set, as well as some missing data.

We therefore choose to look at the unique values in each column.



## We take a look at the unique values in each column


### Global Analysis of Columns

In [ ]:

for col in df:
    print(col + ": ", df[col].unique())
    print("-------------------------------------------------------------------------------------------")



As can be seen some columns only hold meta-data about this dataset. These columns are are: Program, Period, Geo Level, Commodity, Data Item, Domain, and Domain Category.

We propose to drop them.


In [ ]:
df = df.drop(columns=['Program', 'Period', 'Geo Level', 'Commodity', 'Data Item', 'Domain', 'Domain Category'])


Also it is not clear what the CV (%) column is about, and the data within it is all NaN.

Therfore, we also chose to drop this column.


In [ ]:
df = df.drop(columns=['CV (%)'])

We can now start to take a look at each column in more details

In [ ]:
df.head()


### The Year Column


In [ ]:
years = df["Year"]

#### Missing Years

In [ ]:

unique = numpy.sort(years.unique())

min_year = unique[0]
max_year = unique[len(unique) - 1]


print( f"The number of years between {min_year} and {max_year} is: {max_year - min_year}" )
print( f"The number of years recorded in the dataset is: {len(unique)}" )


As we can see, there seems to be 7 years missing in the dataset.

We would like to see which stretches of time correspond to this missing data.

In [ ]:

table = []
for i in range( len(unique) - 1 ):
    if unique[i+1] - unique[i] > 1:
        table.append( [f"{unique[i] + 1}-{unique[i+1]-1}", (unique[i+1] - 1) - (unique[i] + 1)] )

print("The missing years are:")
displayAsTable(table)


We can therefore confirm that that all of the missing years occur between 1909 and 1918.

We can also notice that all of the missing years occur at the beginning of the records, with only one year (1909) preceding them.

Futhuremore, these missing dates are quite old.

Therefore, we can assume that removing the 1909 date from the record is safe, and that it will not affect the trends observed.

In this way we will have a continuous stretch of time.

However, for now we will keep it as we could maybe make use of it later.

#### Distribution of Year Records

We would now like to see how many records exists for each year.

In [ ]:

counts_numpy = numpy.unique(years, return_counts=True)
counts = [counts_numpy[0].tolist(), counts_numpy[1].tolist()]

# Figure Size
fig = plt.figure(figsize =(10, 7))

plt.title("Number of Records per Year")
plt.xlabel("years")
plt.ylabel("number of records")
plt.bar(counts[0], counts[1])
 
# Show Plot
plt.show()


As we can see the number of records per year are somewhat not distributed evenly.

The earlier you go in time, the less records are available.

However, that is not to say that older records are sparse.

Even in the 1920s we can see an average of 500 records per year which is not necessarily that far off from the maximum which is around 2500.

And not that far of from the mean of records per year: 

In [ ]:
print("mean = ", numpy.mean(counts_numpy[1]), "variance = ", numpy.std(counts_numpy[1]))

However, we now have the full justification to get rid of the data points that have a year of 1909 as the number of records associated with this year are vey few compared to other years.

And this is in addition of the previous problems presented on this year.

In [ ]:

indices = df[df["Year"] == 1909].index
df.drop(indices, inplace=True)


## State and State ANSI

We now look at the States and State ANSI columns

## Agricultural District and Agricultural District Code

We will now look and the agricultural district column.

However, before we proceed, we must understand what an agricultural district is.

A district is a logical organisation of land that is genrally under the jurisdiction of a county.

According to the New York state government website, agricultural districts are managed and created by counties, with the guidance of states.

Agricultural districts are therefore, a sub-domain of counties.

This column is quite a complex one since it operates at a finer grain of detail compared to counties (our base line measurment).

Furthuremore, contrary to counties, agricultural districts are highly dependent on local politics and may be created, modified, and destroyed at the whim of local governance.

Therefore, the interpretation of records using this column becomes even more complex when taking into account changes made to those districts.

We will now take a look at the data before continuing our analysis.


In [ ]:

agdistricts = df[["Ag District", "Ag District Code"]]


In [ ]:

unique_districts = agdistricts.groupby(["Ag District", "Ag District Code"]).size().reset_index(name='Freq')
print(unique_districts)


It seems like the agricultural district and its convening code does not match.

Some districts have the same code while others have two different codes for the same district.

It is not really possible to fix that without looking at the data at large. 

It may be the case that the agricultural district "WESTERN" is duplicated because this name exists in multiple states.

However, what is sure, is that it seems like the agricultural district codes seem to be illogically distributed among records.

The agricultutal district column gives us more information than the code one.

For example, we might be able to extract information like the position of the record within the county or state such as: in the "western" part of the county.

For now, we will keep the ag district code but concentrate on the agricultural districts alone. 

However, we will chose to keep the names that are the same in separate rows as they might actually be separate districts.

In [ ]:

unique_districts = unique_districts[["Ag District", "Freq"]]



We will now proceed to analyze the distribution of records among districts.


In [ ]:

print( "Average number of records per district:", numpy.mean(unique_districts["Freq"]) )
print( "Variance of the frequency of records per district:", numpy.var(unique_districts["Freq"]) )
print( "Max frequency:", numpy.max(unique_districts["Freq"]) )
print( "Min frequency:", numpy.min(unique_districts["Freq"]) )



As we can see, while the mean seems acceptanle at 1392, the variance is exeptionally high.

This can also be seen in the difference between the max frequency, and the min frequency.

To understand what is happening, let's look at the different quartiles.

In [ ]:
quants = numpy.quantile(unique_districts["Freq"], [0.1, 0.25, 0.5, 0.75, 0.9, 0.95])
print( f"10% of records have a frequency per district less than {quants[0]}" )
print( f"25% of records have a frequency per district less than {quants[1]}" )
print( f"50% of records have a frequency per district less than {quants[2]}" )
print( f"75% of records have a frequency per district less than {quants[3]}" )
print( f"90% of records have a frequency per district less than {quants[4]}" )
print( f"95% of records have a frequency per district less than {quants[5]}" )

As we can see the majority of records per district are heavily skewed towards the lowe side.

While very few have a very large amount of records.

This distribution seems very uneven.

Let us look at this distribution.

In [ ]:
# Figure Size
fig = plt.figure(figsize =(10, 7))

plt.title("Number of Records per Agricultural District")
plt.xlabel("Agricultural District")
plt.ylabel("Number of Records")
plt.bar(unique_districts["Ag District"], unique_districts["Freq"])
 
# Show Plot
plt.show()

As we can see this distribution is quite uneven.

However, without looking at the full picture, we are unable to make conclusions upon the impact of such a distribution.

We will later analyse those aspects by breaking down this agricultural district column by state counties and years.

This will allow us to understand why there is such a distinct distribution of records.